In [1]:
import os
os.chdir('C:\\Users\\Administrator\\Downloads\\ML Lab')

In [1]:
import sys
sys.path.append(r"C:\Users\PMLS\timeseires")

In [17]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from timeseires.utils.to_split import to_split
from timeseires.utils.multivariate_multi_step import multivariate_multi_step
from timeseires.utils.multivariate_single_step import multivariate_single_step
from timeseires.utils.univariate_multi_step import univariate_multi_step
from timeseires.utils.univariate_single_step import univariate_single_step
from timeseires.utils.CosineAnnealingLRS import CosineAnnealingLRS
from timeseires.callbacks.EpochCheckpoint import EpochCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint
from timeseires.callbacks.TrainingMonitor import TrainingMonitor
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Bidirectional, Add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D,TimeDistributed
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,MaxPooling1D,Concatenate,AveragePooling1D, GlobalMaxPooling1D, Input, SimpleRNN
from tensorflow.keras.models import Sequential,Model
import pandas as pd
import time, pickle
import numpy as np
import tensorflow.keras.backend as K
import tensorflow
from tensorflow.keras.layers import Input, Reshape, Lambda
from tensorflow.keras.layers import Layer, Flatten, LeakyReLU, concatenate, Dense
from tensorflow.keras.regularizers import l2
import glob
import h5py
import matplotlib.pyplot as plt
import os
from keras.callbacks import Callback

In [5]:
#lookback = 24
model = None
start_epoch = 0
time_steps=24
num_features=21

In [7]:
def create_rnn():
    input_data = Input(shape=(time_steps, num_features))
    rnn_layer1 = SimpleRNN(8, return_sequences=True)(input_data)
    rnn_layer2 = SimpleRNN(20)(rnn_layer1)
    x = Flatten()(rnn_layer2)
    output_data = Dense(1)(x)
    model = Model(input_data, output_data)
    return model

In [9]:
model1 = create_rnn()
model1.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 24, 21)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ (None, 24, 8)               │             240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_1 (SimpleRNN)             │ (None, 20)                  │             580 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 20)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              21 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 841 (3.29 KB)

 Trainable params: 841 (3.29 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
tensorflow.keras.utils.plot_model(model1 )

You must install pydot (`pip install pydot`) for `plot_model` to work.


In [19]:
checkpoints = r'C:\Users\PMLS\shahid (ML-lab)\lab9\\E1-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
OUTPUT_PATH = r'C:\Users\PMLS\shahid (ML-lab)\lab9'
FIG_PATH = os.path.sep.join([OUTPUT_PATH,"\history.png"])
JSON_PATH = os.path.sep.join([OUTPUT_PATH,"\history.json"])

<>:3: SyntaxWarning: invalid escape sequence '\h'
<>:4: SyntaxWarning: invalid escape sequence '\h'
<>:3: SyntaxWarning: invalid escape sequence '\h'
<>:4: SyntaxWarning: invalid escape sequence '\h'
C:\Users\PMLS\AppData\Local\Temp\ipykernel_10972\4261185172.py:3: SyntaxWarning: invalid escape sequence '\h'
  FIG_PATH = os.path.sep.join([OUTPUT_PATH,"\history.png"])
C:\Users\PMLS\AppData\Local\Temp\ipykernel_10972\4261185172.py:4: SyntaxWarning: invalid escape sequence '\h'
  JSON_PATH = os.path.sep.join([OUTPUT_PATH,"\history.json"])


In [21]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1,TrainingMonitor1]

In [23]:
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model =create_rnn()
    opt = Adam(1e-3)
    model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] compiling model...


In [27]:
import os
path_dataset =r'C:\Users\PMLS\shahid (ML-lab)\lab7'
path_tr = os.path.join(path_dataset, 'AEP_train.csv')
df_tr = pd.read_csv(path_tr)
train_set = df_tr.iloc[:].values
path_v = os.path.join(path_dataset, 'AEP_validation.csv')
df_v = pd.read_csv(path_v)
validation_set = df_v.iloc[:].values 
path_te = os.path.join(path_dataset, 'AEP_test.csv')
df_te = pd.read_csv(path_te)
test_set = df_te.iloc[:].values 

path_scaler = os.path.join(path_dataset, 'AEP_Scaler.pkl')
scaler         = pickle.load(open(path_scaler, 'rb'))

train_set.shape, validation_set.shape, test_set.shape

C:\Users\PMLS\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


((84907, 21), (24259, 21), (12130, 21))

In [29]:
time_steps=24
num_features=21

In [31]:
start = time.time()
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=1)
validation_X, validation_y = univariate_multi_step(validation_set, time_steps, target_col=0,target_len=1)
test_X, test_y = univariate_multi_step(test_set, time_steps, target_col=0,target_len=1)
print('Time Consumed', time.time()-start, "sec")

Time Consumed 0.36496400833129883 sec


In [33]:
epochs = 60
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,verbose = verbose)

Epoch 1/60
2644/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0785 - mae: 0.0785 - mape: 5439.2632
Epoch 1: val_loss improved from inf to 0.01905, saving model to C:\Users\PMLS\shahid (ML-lab)\lab9\\E1-cp-0001-loss0.02.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 19s 6ms/step - loss: 0.0784 - mae: 0.0784 - mape: 5429.9839 - val_loss: 0.0190 - val_mae: 0.0190 - val_mape: 8.3598
Epoch 2/60
2647/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0159 - mae: 0.0159 - mape: 73.0206
Epoch 2: val_loss improved from 0.01905 to 0.01401, saving model to C:\Users\PMLS\shahid (ML-lab)\lab9\\E1-cp-0002-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - loss: 0.0159 - mae: 0.0159 - mape: 73.2962 - val_loss: 0.0140 - val_mae: 0.0140 - val_mape: 7.1730
Epoch 3/60
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0134 - mae: 0.0134 - mape: 29.1331
Epoch 3: val_loss improved from 0.01401 to 0.01204, saving model to C:\Users\PMLS\shahid (ML-lab)\lab9\\E1-cp-0003-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - loss: 0.0134 - mae: 0.0134 - mape: 29.1307 - val_loss: 0.0120 - val_mae: 0.0120 - val_mape: 5.1822
Epoch 4/60
2650/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0123 - mae: 0.0123 - mape: 14.5447
Epoch 4: val_loss improved from 0.01204 to 0.01014, saving model to C:\Users\PMLS\shahid (ML-lab)\lab9\\E1-cp-0004-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - loss: 0.0123 - mae: 0.0123 - mape: 14.7325 - val_loss: 0.0101 - val_mae: 0.0101 - val_mape: 4.7015
Epoch 5/60
2652/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0109 - mae: 0.0109 - mape: 95.1299
Epoch 5: val_loss improved from 0.01014 to 0.00999, saving model to C:\Users\PMLS\shahid (ML-lab)\lab9\\E1-cp-0005-loss0.01.h5


...

2653/2653 ━━━━━━━━━━━━━━━━━━━━ 16s 6ms/step - loss: 0.0083 - mae: 0.0083 - mape: 196.7921 - val_loss: 0.0079 - val_mae: 0.0079 - val_mape: 3.8113
Epoch 18/60
2648/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0082 - mae: 0.0082 - mape: 424.6689
Epoch 18: val_loss improved from 0.00793 to 0.00772, saving model to C:\Users\PMLS\shahid (ML-lab)\lab9\\E1-cp-0018-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - loss: 0.0082 - mae: 0.0082 - mape: 424.2048 - val_loss: 0.0077 - val_mae: 0.0077 - val_mape: 3.4923
Epoch 19/60
2643/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0084 - mae: 0.0084 - mape: 984.7134
Epoch 19: val_loss did not improve from 0.00772
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - loss: 0.0084 - mae: 0.0084 - mape: 981.5917 - val_loss: 0.0081 - val_mae: 0.0081 - val_mape: 3.5956
Epoch 20/60
2651/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0082 - mae: 0.0082 - mape: 156.6363
Epoch 20: val_loss did not improve from 0.00772
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - loss: 0.0082 - mae: 0.0082 - mape: 156.7106 - val_loss: 0.0081 - val_mae: 0.0081 - val_mape: 3.4463
Epoch 21/60
2648/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0082 - mae: 0.0082 - mape: 914.7872
Epoch 21: val_loss did not improve from 0.00772
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - loss: 0.0082 - mae: 0.0082 - mape: 913.3567 - val_loss: 0.0077 - val_

2653/2653 ━━━━━━━━━━━━━━━━━━━━ 16s 6ms/step - loss: 0.0081 - mae: 0.0081 - mape: 379.8436 - val_loss: 0.0074 - val_mae: 0.0074 - val_mape: 3.6699
Epoch 23/60
2647/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0080 - mae: 0.0080 - mape: 112.5134
Epoch 23: val_loss did not improve from 0.00736
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - loss: 0.0080 - mae: 0.0080 - mape: 113.1074 - val_loss: 0.0090 - val_mae: 0.0090 - val_mape: 4.9998
Epoch 24/60
2646/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0080 - mae: 0.0080 - mape: 150.3775
Epoch 24: val_loss did not improve from 0.00736
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - loss: 0.0080 - mae: 0.0080 - mape: 150.5259 - val_loss: 0.0077 - val_mae: 0.0077 - val_mape: 3.5549
Epoch 25/60
2647/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0079 - mae: 0.0079 - mape: 359.7134
Epoch 25: val_loss did not improve from 0.00736
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - loss: 0.0079 - mae: 0.0079 - mape: 359.5497 - val_loss: 0.0075 - val_

2653/2653 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - loss: 0.0080 - mae: 0.0080 - mape: 109.2833 - val_loss: 0.0073 - val_mae: 0.0073 - val_mape: 3.3635
Epoch 29/60
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0078 - mae: 0.0078 - mape: 143.9693
Epoch 29: val_loss did not improve from 0.00728
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - loss: 0.0078 - mae: 0.0078 - mape: 143.9668 - val_loss: 0.0078 - val_mae: 0.0078 - val_mape: 3.6397
Epoch 30/60
2651/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0079 - mae: 0.0079 - mape: 443.4020
Epoch 30: val_loss did not improve from 0.00728
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 16s 6ms/step - loss: 0.0079 - mae: 0.0079 - mape: 443.2482 - val_loss: 0.0082 - val_mae: 0.0082 - val_mape: 3.6160
Epoch 31/60
2649/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0077 - mae: 0.0077 - mape: 230.9211
Epoch 31: val_loss did not improve from 0.00728
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - loss: 0.0077 - mae: 0.0077 - mape: 230.8129 - val_loss: 0.0078 - val_

KeyboardInterrupt: 

In [40]:

#model = load_model(r'C:\Users\PMLS\shahid (ML-lab)\lab9\E1-cp-0028-loss0.01.h5')

model = load_model(r'C:\Users\PMLS\shahid (ML-lab)\lab9\E1-cp-0028-loss0.01.h5',
                   custom_objects={'mae': mean_absolute_error})

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

379/379 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Mean Absolute Error (MAE): 117.19
Median Absolute Error (MedAE): 91.19
Mean Squared Error (MSE): 24762.91
Root Mean Squared Error (RMSE): 157.36
Mean Absolute Percentage Error (MAPE): 0.8 %
Median Absolute Percentage Error (MDAPE): 0.63 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)


In [46]:
checkpoints = r'C:\Users\PMLS\shahid (ML-lab)\lab9\E2-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
model=r'C:\Users\PMLS\shahid (ML-lab)\lab9\E1-cp-0028-loss0.01.h5'
start_epoch= 54

In [88]:
from keras.metrics import mean_absolute_error as mae
from keras.metrics import mean_absolute_percentage_error as mape
from keras.models import load_model
import numpy as np
from keras import backend as K

# Define your model path (replace with your actual path)
model_path = r'C:\Users\PMLS\shahid (ML-lab)\lab9\E1-cp-0028-loss0.01.h5'

# construct the callback to save only the *best* model to disk
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
TrainingMonitor1 = TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1, TrainingMonitor1]

# if there is no specific model checkpoint supplied, initialize new model
if model_path is None:
    print("[INFO] compiling model...")
    model = PC.build(time_steps=24, num_features=21, reg=0.0005)
    opt = Adam(1e-3)
    model.compile(loss='mean_absolute_error', 
                 optimizer=opt, 
                 metrics=["mean_absolute_error", "mean_absolute_percentage_error"])
else:
    print("[INFO] loading {}...".format(model_path))
    try:
        # First try loading with custom objects
        model = load_model(model_path, custom_objects={
            'mae': mae,
            'mape': mape,
            'mean_absolute_error': mae,
            'mean_absolute_percentage_error': mape
        })
    except Exception as e:
        print(f"First load attempt failed: {e}")
        try:
            # Fallback to loading without custom objects
            model = load_model(model_path)
        except Exception as e:
            print(f"Second load attempt failed: {e}")
            try:
                # Final fallback - load without compilation
                model = load_model(model_path, compile=False)
                model.compile(loss='mean_absolute_error',
                            optimizer=Adam(1e-4),
                            metrics=[mae, mape])
            except Exception as e:
                print(f"Final load attempt failed: {e}")
                raise RuntimeError("Could not load model with any method")

    # CORRECTED LEARNING RATE UPDATE APPROACH
    try:
        # Get the actual learning rate variable
        lr_var = model.optimizer.learning_rate
        if not isinstance(lr_var, str):  # Only proceed if it's not a string
            old_lr = float(K.get_value(lr_var))
            print("[INFO] old learning rate: {}".format(old_lr))
            
            # Set new learning rate - using Keras backend properly
            K.set_value(lr_var, 1e-4)
            
            new_lr = float(K.get_value(lr_var))
            print("[INFO] new learning rate: {}".format(new_lr))
        else:
            print("[WARNING] Learning rate appears to be a string, cannot update")
    except Exception as e:
        print(f"[WARNING] Could not update learning rate: {e}")
        # Alternative approach if the above fails
        try:
            model.optimizer.learning_rate.assign(1e-4)
            print("[INFO] Updated learning rate using direct assignment")
        except Exception as e:
            print(f"[WARNING] Failed to update learning rate completely: {e}")

[INFO] loading C:\Users\PMLS\shahid (ML-lab)\lab9\E1-cp-0028-loss0.01.h5...
[WARNING] Could not update learning rate: module 'keras.api.backend' has no attribute 'get_value'
[INFO] Updated learning rate using direct assignment


In [90]:
epochs = 10
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,
                        verbose = verbose)

Epoch 1/10


NotImplementedError: numpy() is only available when eager execution is enabled.

In [19]:

model = load_model(r'C:\Users\Administrator\Downloads\ML Lab\checkpoint\ML Lab\lab9\E2-cp-0006-loss0.01.h5')

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

379/379 [==============================] - 1s 1ms/step
Mean Absolute Error (MAE): 144.75
Median Absolute Error (MedAE): 113.65
Mean Squared Error (MSE): 36189.68
Root Mean Squared Error (RMSE): 190.24
Mean Absolute Percentage Error (MAPE): 0.98 %
Median Absolute Percentage Error (MDAPE): 0.79 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)
